In [2]:
!pip install kaggle


In [10]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Install dependencies
!pip install -U -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17372, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 17372 (delta 42), reused 20 (delta 20), pack-reused 17313 (from 3)
Receiving objects: 100% (17372/17372), 16.26 MiB | 10.23 MiB/s, done.
Resolving deltas: 100% (11907/11907), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s et

In [3]:
from google.colab import files
files.upload()  # Pilih file kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ayamgoreng21","key":"71fae8e61c7c500a0d34219a11b7fb5c"}'}

In [4]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')


In [5]:
!kaggle datasets download -d aneesarom/rider-with-helmet-without-helmet-number-plate


Dataset URL: https://www.kaggle.com/datasets/aneesarom/rider-with-helmet-without-helmet-number-plate
License(s): apache-2.0
rider-with-helmet-without-helmet-number-plate.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip rider-with-helmet-without-helmet-number-plate -d /content


Archive:  rider-with-helmet-without-helmet-number-plate.zip
replace /content/classes.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/classes.txt    
replace /content/coco128.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/coco128.yaml   
replace /content/train/images/new1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/train/images/new1.jpg  
replace /content/train/images/new10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/train/images/new10.jpg  
replace /content/train/images/new100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/train/images/new100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/train/images/new100.jpg  
  inflating: /content/train/images/new101.jpg  
  inflating: /content/train/images/new102.jpg  
  inflating: /content/train/images/new103.jpg  
  inflating: /content/train/images/new106.jpg  
  inflating: /content/train/images/n

In [7]:
import os

# Tentukan path folder output
output_dir = '/content/output_labels'

# Cek jika folder output tidak ada, maka buat folder
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Folder {output_dir} telah dibuat.")
else:
    print(f"Folder {output_dir} sudah ada.")


Folder /content/output_labels sudah ada.


In [8]:
import os

# Tentukan path folder output
output_dir = '/content/train/broken_images'

# Cek jika folder output tidak ada, maka buat folder
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Folder {output_dir} telah dibuat.")
else:
    print(f"Folder {output_dir} sudah ada.")


Folder /content/train/broken_images sudah ada.


In [9]:
import os
from PIL import Image, UnidentifiedImageError
import shutil

def move_broken_images(image_dir, output_dir):
    for image_name in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_name)
        try:
            # Verifikasi gambar
            img = Image.open(image_path)
            img.verify()
        except UnidentifiedImageError:
            # Pindahkan gambar yang rusak ke folder terpisah
            print(f"Memindahkan gambar rusak: {image_name}")
            shutil.move(image_path, os.path.join(output_dir, image_name))

# Misalnya, pindahkan gambar rusak ke folder 'broken_images'
move_broken_images('/content/train/images', '/content/train/broken_images')


Memindahkan gambar rusak: new128.jpg


In [10]:
import os
from PIL import Image

def convert_to_yolo_format(image_dir, label_dir, output_dir):
    # Loop over each image in the directory
    for image_name in os.listdir(image_dir):
        if image_name.endswith(".jpg"):
            # Load image
            image_path = os.path.join(image_dir, image_name)
            img = Image.open(image_path)
            width, height = img.size

            # Load corresponding label text file
            label_file = image_name.replace(".jpg", ".txt")  # Ganti .jpg dengan .txt
            label_path = os.path.join(label_dir, label_file)

            try:
                with open(label_path, 'r') as file:
                    lines = file.readlines()
            except FileNotFoundError:
                print(f"Label file {label_file} not found.")
                continue  # Skip this image if the label file is missing

            # Create the output label file in YOLO format
            output_label_file = os.path.join(output_dir, label_file)  # Make sure output dir exists
            with open(output_label_file, 'w') as out_file:
                for line in lines:
                    # Assumes that labels are already in YOLO format, write them directly
                    out_file.write(line)

# Call function to convert the labels
convert_to_yolo_format('/content/train/images', '/content/train/labels', '/content/output_labels')


In [11]:
%cd /content/yolov5


/content/yolov5


In [12]:
!python3 train.py --img 640 --batch 16 --epochs 50 --data data/helmet_detection.yaml --weights yolov5s.pt --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-12 07:55:59.920101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744444559.941791    3304 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744444559.949353    3304 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&

In [13]:
import random
import os
from IPython.display import Image, display
import torch
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [17]:
import os
import random
import shutil

# Set path untuk folder train dan test
train_dir = '/content/train/images'  # Ganti dengan path folder train kamu
label_dir = '/content/train/labels'  # Path folder label
test_dir = '/content/test'    # Ganti dengan path folder test yang ingin dibuat

# Membuat folder test jika belum ada
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Fungsi untuk menyalin file
def copy_files(file_list, dest_folder):
    for file in file_list:
        shutil.copy(file, dest_folder)

# Ambil semua file gambar dan anotasi dari folder train
image_extensions = ['.jpg', '.jpeg', '.png']
image_paths = []
label_paths = []

for root, dirs, files in os.walk(train_dir):
    for file in files:
        if any(file.endswith(ext) for ext in image_extensions):
            image_paths.append(os.path.join(root, file))
            label_file = file.rsplit('.', 1)[0] + '.txt'  # Pastikan label menggunakan ekstensi .txt
            label_paths.append(os.path.join(label_dir, label_file))

# Tentukan jumlah data yang akan dipindahkan ke test (misal 20% dari total data)
test_size = int(0.2 * len(image_paths))  # 20% data untuk test

# Pilih file secara acak untuk test
test_images = random.sample(image_paths, test_size)
test_labels = [label for img, label in zip(image_paths, label_paths) if img in test_images]

# Salin file gambar dan anotasi ke folder test
test_image_dir = os.path.join(test_dir, 'images')
test_label_dir = os.path.join(test_dir, 'labels')

# Buat folder untuk test jika belum ada
os.makedirs(test_image_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)

# Salin gambar dan label yang terpilih
copy_files(test_images, test_image_dir)
copy_files(test_labels, test_label_dir)

print(f"Berhasil memindahkan {len(test_images)} gambar dan {len(test_labels)} label ke folder test.")


Berhasil memindahkan 20 gambar dan 20 label ke folder test.


In [32]:
import torch
from pathlib import Path
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Set path to the trained model
model_path = '/content/yolov5/runs/train/exp2/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Set path to test images folder
test_image_dir = '/content/test/images'

# Get a random image from the test folder
image_paths = list(Path(test_image_dir).glob('*.jpg'))
if not image_paths:
    print(f"No jpg images found in {test_image_dir}")
    exit()

random_image = random.choice(image_paths)
print(f"Testing on: {random_image}")
print(f"File exists: {random_image.exists()}")
print(f"File size: {random_image.stat().st_size} bytes")

# Read the image using OpenCV
image = cv2.imread(str(random_image))
if image is None:
    print("Image cannot be read.")
else:
    print(f"Image dimensions: {image.shape}")
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Make predictions using YOLOv5 model
    results = model(image_rgb)
    print(f"Number of detections: {len(results.xyxy[0])}")

    # Display the image with bounding boxes using matplotlib
    fig, ax = plt.subplots(1, figsize=(12, 12))

    # Show the original image
    ax.imshow(image_rgb)

    # Draw bounding boxes
    if len(results.xyxy[0]) == 0:
        print("No objects detected.")
        plt.title("No objects detected")
    else:
        for detection in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = detection.cpu().numpy()
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            w, h = x2 - x1, y2 - y1

            # Draw rectangle and confidence score
            ax.add_patch(plt.Rectangle((x1, y1), w, h, fill=False, color='red', linewidth=3))
            ax.text(x1, y1, f'{conf:.2f}', color='white', fontsize=12, weight='bold',
                    bbox=dict(facecolor='red', alpha=0.5))

        plt.title(f"Found {len(results.xyxy[0])} objects")

    plt.axis('off')
    # Force display
    plt.tight_layout()
    plt.savefig('/content/detection_output.jpg')  # Save image to file
    plt.show()

    print("Plot should be displayed. If not visible, check the saved image at /content/detection_output.jpg")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-12 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Testing on: /content/test/images/new118.jpg
File exists: True
File size: 1060110 bytes
Image dimensions: (1052, 518, 3)
Number of detections: 3
Plot should be displayed. If not visible, check the saved image at /content/detection_output.jpg


In [37]:
import torch
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Set path to the trained model
model_path = '/content/yolov5/runs/train/exp2/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Set path to test images folder
test_image_dir = '/content/val/images'
test_label_dir = '/content/val/labels'  # Assuming this folder contains label files

# Get a list of all test images
image_paths = list(Path(test_image_dir).glob('*.jpg'))  # or '*.png' if needed
label_paths = [Path(test_label_dir) / (img.stem + '.txt') for img in image_paths]

# Initialize lists for ground truth and predictions
y_true = []  # Ground truth
y_pred = []  # Predictions

# Define class names (optional: modify as needed)
class_names = ['Class 0', 'Class 1', 'Class 2']

# Iterate over each image and label
for image_path, label_path in zip(image_paths, label_paths):
    # Read the image using OpenCV
    image = cv2.imread(str(image_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Make predictions using YOLOv5 model
    results = model(image_rgb)

    # Get the predicted classes
    pred_classes = results.xyxy[0][:, -1].cpu().numpy().astype(int)  # Predicted classes

    # Read ground truth labels from the .txt file
    with open(label_path, 'r') as file:
        ground_truth = [int(line.split()[0]) for line in file.readlines()]

    # Only append to lists if both ground truth and predictions are present
    if len(pred_classes) > 0:  # Check if predictions are made
        if len(ground_truth) == len(pred_classes):  # If the number of predictions matches the ground truth
            y_true.extend(ground_truth)
            y_pred.extend(pred_classes)
        else:
            print(f"Mismatched number of predictions and ground truth for {image_path}. Skipping this image.")
    else:
        print(f"No predictions for {image_path}, skipping.")

# Check if y_true and y_pred have the same length
if len(y_true) != len(y_pred):
    print(f"Length mismatch: y_true={len(y_true)}, y_pred={len(y_pred)}")
else:
    # Now, calculate the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot confusion matrix using seaborn heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

    # Optionally, you can print the confusion matrix as well
    print("Confusion Matrix:")
    print(cm)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-12 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.

Mismatched number of predictions and ground truth for /content/val/images/new74.jpg. Skipping this image.
Mismatched number of predictions and ground truth for /content/val/images/new41.jpg. Skipping this image.
Mismatched number of predictions and ground truth for /content/val/images/new9.jpg. Skipping this image.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am

Mismatched number of predictions and ground truth for /content/val/images/new4.jpg. Skipping this image.
Mismatched number of predictions and ground truth for /content/val/images/new113.jpg. Skipping this image.
Mismatched number of predictions and ground truth for /content/val/images/new127.jpg. Skipping this image.
Mismatched number of predictions and ground truth for /content/val/images/new49.jpg. Skipping this image.


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Confusion Matrix:
[[4 0 1 3]
 [0 6 1 1]
 [3 2 4 4]
 [1 0 7 5]]


In [42]:
import torch
import random
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import os
import numpy as np

# Set path to the trained model
model_path = '/content/yolov5/runs/train/exp2/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Get class names from the model
class_names = model.names if hasattr(model, 'names') else [f'Class {i}' for i in range(1000)]
print(f"Available classes: {class_names}")

def plot_pred_image(image_name):
    # Set the image path
    image_path = f'/content/val/images/{image_name}.jpg'

    # Read the image using OpenCV
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Make predictions using YOLOv5 model
    results = model(image_rgb)

    # Create a figure for displaying
    plt.figure(figsize=(12, 10))
    plt.imshow(image_rgb)

    # Check if any detections were found
    if len(results.xyxy[0]) == 0:
        plt.title(f'No detections for {image_name}')
    else:
        # Draw bounding boxes for each detection
        for detection in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls_id = detection.cpu().numpy()
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            cls_id = int(cls_id)

            # Get class name
            cls_name = class_names[cls_id] if cls_id < len(class_names) else f'Class {cls_id}'

            # Create rectangle patch
            from matplotlib.patches import Rectangle
            rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2,
                             edgecolor='r', facecolor='none')
            plt.gca().add_patch(rect)

            # Add label with class name and confidence
            plt.text(x1, y1-10, f'{cls_name}: {conf:.2f}',
                    color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.7))

        plt.title(f'Detected {len(results.xyxy[0])} objects in {image_name}')

    plt.axis('off')
    plt.tight_layout()

    # Save the image with bounding boxes
    output_path = f'/content/detection_{image_name}.jpg'
    plt.savefig(output_path, bbox_inches='tight')
    print(f"Detection image saved to {output_path}")

    plt.show()
    return True

def get_random_image_name(image_dir='/content/val/images'):
    # Get all image filenames in the directory
    image_paths = list(Path(image_dir).glob('*.jpg'))

    if not image_paths:
        print(f"No JPG images found in {image_dir}")
        return None

    # Select a random image
    random_image = random.choice(image_paths)
    image_name = random_image.stem  # Get the filename without extension
    print(f"Selected random image: {image_name}")

    return image_name

# Get a random image name
random_image_name = get_random_image_name()

if random_image_name:
    # Display the prediction for the random image
    print('\n--- Prediction with Bounding Boxes ---')
    plot_pred_image(image_name=random_image_name)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-4-12 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Available classes: {0: 'with helmet', 1: 'without helmet', 2: 'rider', 3: 'number plate'}
Selected random image: new73

--- Prediction with Bounding Boxes ---
Detection image saved to /content/detection_new73.jpg
